Putting all together what we have learned to far. Theirfor we use more complex data from [Kaggle](www.kaggle.com).

Kaggle is a website to find datasets, learn more about nn and stuff and compete in competitions!

For this example we are using this [dataset](https://www.kaggle.com/datasets/mirichoi0218/insurance)

on [github](https://github.com/stedy/Machine-Learning-with-R-datasets)



In [66]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras as ks
import os
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.getLogger('absl').setLevel(logging.ERROR)

In [67]:
# run wget https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/refs/heads/master/insurance.csv
insurance_dataset_original = pd.read_csv('insurance.csv')

In [68]:
insurance_dataset_original

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


First step is to normalize the dataset

1) age is of type
2) sex is of type
3) bmi is of type
4) children is of type
5) smoker is of type
6) region is of type
7) charges is of type

In [69]:
insurance_dataset = insurance_dataset_original.copy(deep=True)

In [70]:
# make data more easy
def round_3(n):
    if n in [0,1,2]:
        return -n
    elif n in [3,4,5,6,7]:
        return -n+5
    elif n in [8,9]:
        return -n+10

def round_4(n):
    if n in [0,1]:
        return -n
    elif n in [2,3,4]:
        return -n+3
    elif n in [5,6,7]:
        return -n+6
    elif n in [8,9]:
        return -n+10

def round_children(n):
    if n == 0:
        return 'none'
    elif n in [1,2]:
        return 'some'
    elif n > 2:
        return 'many'


age_dataset = insurance_dataset['age']
insurance_dataset['age'] = age_dataset.apply(lambda x: x+round_3(x%10))

bmi_dataset = insurance_dataset['bmi']
insurance_dataset['bmi'] = bmi_dataset.apply(lambda x: round(x)+round_4(round(x)%10))

children_dataset = insurance_dataset['children'].infer_objects()
insurance_dataset['children'] = children_dataset.apply(lambda x: round_children(x))

charges_dataset = pd.to_numeric(insurance_dataset['charges'])
insurance_dataset['charges'] = charges_dataset.apply(lambda x: int(x/1000))

insurance_dataset

,age,sex,bmi,children,smoker,region,charges
0,20,female,30,none,yes,southwest,16
1,20,male,33,some,no,southeast,1
2,30,male,33,many,no,southeast,4
3,35,male,23,none,no,northwest,21
4,30,male,30,none,no,northwest,3
...,...,...,...,...,...,...,...
1333,50,male,30,many,no,northwest,10
1334,20,female,33,none,no,northeast,2
1335,20,female,36,none,no,southeast,1
1336,20,female,26,none,no,southwest,2


In [72]:
insurance_dataset_one_hot = pd.get_dummies(insurance_dataset)
insurance_dataset_one_hot

,age,bmi,charges,sex_female,sex_male,children_many,children_none,children_some,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,20,30,16,True,False,False,True,False,False,True,False,False,False,True
1,20,33,1,False,True,False,False,True,True,False,False,False,True,False
2,30,33,4,False,True,True,False,False,True,False,False,False,True,False
3,35,23,21,False,True,False,True,False,True,False,False,True,False,False
4,30,30,3,False,True,False,True,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30,10,False,True,True,False,False,True,False,False,True,False,False
1334,20,33,2,True,False,False,True,False,True,False,True,False,False,False
1335,20,36,1,True,False,False,True,False,True,False,False,False,True,False
1336,20,26,2,True,False,False,True,False,True,False,False,False,False,True


In [88]:
# create test and training data 30 / 70
perc = int(len(insurance_dataset_one_hot.age) * (7/10))
insurance_train = insurance_dataset_one_hot[0:perc]
insurance_test = insurance_dataset_one_hot[perc:]
len(insurance_train), len(insurance_test)

(936, 402)